# VacationPy

In [1]:
#Import gmaps, pandas, os, requests, and json and config gkey
import requests
import json
import os
import pandas as pd
import gmaps
from configure import gkey

gmaps.configure(api_key=gkey)

In [2]:
#Import and display list of cities saved in Part I
cities_list=pd.read_csv("weather_data.csv")
cities_list

,City ID,City Name,Country Code,Date,Latitude,Longitude,Humidity %,Temperature (F),Wind Speed (mph),Cloudiness %
0,780687,Berlevåg,NO,05/05/20,70.86,29.09,59,276.15,10.30,100
1,2075265,Busselton,AU,05/05/20,-33.65,115.33,98,282.36,13.12,99
2,2640377,Penzance,GB,05/05/20,50.12,-5.54,100,283.71,8.70,98
3,4030556,Rikitea,PF,05/05/20,-23.12,-134.97,62,296.24,4.48,100
4,1819729,Hong Kong,HK,05/05/20,22.29,114.16,78,300.57,2.68,100
...,...,...,...,...,...,...,...,...,...,...
557,2629833,Husavik,IS,05/05/20,66.04,-17.34,34,285.15,14.90,0
558,2119283,Zyryanka,RU,05/05/20,65.75,150.85,98,270.17,2.03,83
559,3115739,Narón,ES,05/05/20,43.52,-8.15,68,289.51,5.70,100
560,2021031,Kyren,RU,05/05/20,51.68,102.14,82,277.46,1.80,100


In [3]:
# Create latitude and longitude in locations
locations = cities_list[["Latitude", "Longitude"]]
humidity = cities_list["Humidity %"].astype(float)
max_humidity=humidity.max()

In [4]:
# Plot heatmap

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, zoom_level=1,center=(15,25))

# Plot heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=max_humidity,
    point_radius=3)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
#Select data with perfect weather
perfect_weather_df = cities_list.loc[(cities_list["Temperature (F)"] > 70) & (cities_list["Temperature (F)"] < 80) & (cities_list["Cloudiness %"] == 0) & (cities_list["Wind Speed (mph)"] <15), :]

perfect_weather_df = perfect_weather_df.dropna(how='any')
perfect_weather_df.reset_index(inplace=True)

perfect_weather_df

,index,City ID,City Name,Country Code,Date,Latitude,Longitude,Humidity %,Temperature (F),Wind Speed (mph),Cloudiness %


In [6]:
#Create column shell to add hotel names
perfect_weather_df["Hotel Name"] = ""


In [7]:
#Set parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": gkey
}

# Loop through data to pull latitude and longitude
for index, row in perfect_weather_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Search for hotels based on latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Request data
    name_address = requests.get(base_url, params=params)

    # Convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Add try and except to incorporate missing data
    try:
        perfect_weather_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [8]:
# Add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_location = [info_box_template.format(**row) for index, row in perfect_weather_df.iterrows()]
locations = perfect_weather_df[["Latitude", "Longitude"]]

In [9]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=1,center=(15,25))

# Create hotel symbol to add to map
hotel_with_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in perfect_weather_df.iterrows()]
)

# Add layers to map
fig.add_layer(heat_layer)
fig.add_layer(hotel_with_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…